In [6]:
import torch
from models import skipgram
import pandas as pd
import os
from util import artifacts
import tokenization

import importlib
importlib.reload(artifacts)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cameron/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'util.artifacts' from '/Users/cameron/Documents/mlx/hackernews/util/artifacts.py'>

In [7]:
loaded_artifacts = artifacts.load_artifacts()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [ ]:
state_dict = loaded_artifacts['state_dict']
vocab = loaded_artifacts['vocab'].set_index('token')

model = skipgram.Model(vocab.size + 1, skipgram.EMBEDDING_DIM)

# #find the n most similar words
def most_similar(word, n=5):
    word_idx = tokenization.getIdFromToken(vocab, word)
    if (word_idx == tokenization.getIdFromToken(vocab, 'unk')):
        raise ValueError("Word not found in vocabulary")
    A =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
    word_similarities = []
    for token, id in vocab.iterrows():
        word_idx = tokenization.getIdFromToken(vocab, token)
        B =  state_dict['embeddings.weight'][word_idx].unsqueeze(0)
        cosine_similarity = torch.nn.functional.cosine_similarity(A, B, dim=1)
        word_similarities.append((token, cosine_similarity.item()))
    word_similarities = sorted(word_similarities, key=lambda x: x[1], reverse=True)
    return word_similarities[:n]

def similarity(word1, word2):
    word1_idx = tokenization.getIdFromToken(vocab, word1)
    if (word1_idx == tokenization.getIdFromToken(vocab, 'unk')):
        raise ValueError("Word not found in vocabulary")
    A =  state_dict['embeddings.weight'][word1_idx].unsqueeze(0)
    word2_idx = tokenization.getIdFromToken(vocab, word2)
    if (word2_idx == tokenization.getIdFromToken(vocab, 'unk')):
        raise ValueError("Word not found in vocabulary")
    B =  state_dict['embeddings.weight'][word2_idx].unsqueeze(0)
    cosine_similarity = torch.nn.functional.cosine_similarity(A, B, dim=1)
    return cosine_similarity


tensor([0.1222])